# Dragon vs Jabbing Style

This question started the whole project.
I was wondering which of the two style paths has a higher damage output. Assuming that all my attacks hit clearly the answer was the jabbing style. However, most of the jabbing damage comes from the later attacks, which are less likely to hit, whereas dragon style leads to more damage on your first attack. 

Then I decided to challenge myself, because I did not want to do the computation with a fixed armor class, but (more realistically I hoped) specifying the challenge rating of the opponent averaging over the distribution of armor classes. This functionality has been exported to the monster.py module. 

This notebook is an example computation of my Level 10 Brawler (which is not particularly optimized for damage).

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
from dotenv import dotenv_values

from monsters import Monsters

In [2]:
path = dotenv_values()['MONSTER_PATH']

The variable $m$ contains the whole monster database with auxiliarly methods to compute the necessary statistics

In [3]:
m = Monsters(path)

/home/tim/Projekte/Pathfinder/monsters.py:55: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self.cr = self.groupby('CR').agg(['mean', 'std'])


The necessary properties of my brawler.
I also assume that I have the feats weapon focus (unarmed strike) and elemental fists
I did the computations without power attack. If desired this can be easily adapted. Also I did the computations with Brawler's flurry active

In [4]:
str_mod = 4
bab = 10
magic_weapon = 2
weapon_focus = 1
mali = 2

In [5]:
offset = str_mod + bab + magic_weapon + weapon_focus - mali
ar_offsets = [offset]*2 + [offset - 5]*2

I assume challenge rating 12 (we have 3 mythic tiers and are 5 people in the party). Maybe even 13 would make sense

In [6]:
cr = 12

## Dragon style setup
We need the damage roles for the dragon style feat chain

In [7]:
dmg_rolls = [
    f'1d10+3d6+{2*str_mod+magic_weapon}', #Brawler damage die + elemental fist (as monk of the four winds) + 2x str (bc of Dragon Ferocity)
    f'1d10+{int(1.5*str_mod+magic_weapon)}', #1.5x str bc of Dragon Ferocity
    f'1d10+{int(1.5*str_mod+magic_weapon)}', # as above
    f'1d10+{int(1.5*str_mod+magic_weapon)}', # as above
]

Probability to hit:

In [8]:
for i, p in enumerate(m.hit_probability(ar_offsets, cr)):
    print(f'{i+1}. attack: {p:.3f}')

1. attack: 0.468
2. attack: 0.468
3. attack: 0.223
4. attack: 0.223


This was worse than I expected :-D

### Damage estimation (Dragon style)

In [9]:
total_mean = 0.
total_std = 0.
for i, (mean, std) in enumerate(m.expected_damage(dmg_rolls, ar_offsets, cr)):
    print(f'{i+1}. attack: {mean:5.2f} +/- {std:1.2f}')
    total_mean += mean
    total_std += std**2
    

print(f'\nTotal: {total_mean:9.2f} +/- {np.sqrt(total_std):.2f}')

1. attack: 12.16 +/- 2.82
2. attack:  6.31 +/- 1.96
3. attack:  3.01 +/- 1.36
4. attack:  3.01 +/- 1.36

Total:     24.48 +/- 3.93


# Jabbing style
Jabbing style is more involved, because the later damage depends upon the previous hits. So I will for now only compute the mean damage (and even there might be an error regarding the probabilities, so pleas correct me :-)

In [10]:
p_hit_once = 0.
p_hit_more_than_once = 0.
total_mean = 0.

print('            Dmg (p(once) / p(>once))')

for i, p in enumerate(m.hit_probability(ar_offsets, cr)):
    mean = p*(str_mod + magic_weapon + 5.5 + p_hit_once*7 + p_hit_more_than_once*14) # str + enahncement + damage dice
    if i == 0: mean += p*3.5 # elemental fist on first attack
    print(f'{i+1}. attack: {mean:5.2f} ({p_hit_once:02.02f} / {p_hit_more_than_once:02.02f})')
    total_mean += mean
    
    p_hit_more_than_once += (1 - p_hit_more_than_once)*p_hit_once*p
    p_hit_once += (1 - p_hit_once)*p
    
    
print(f'\nTotal: {total_mean:9.2f}')

            Dmg (p(once) / p(>once))
1. attack:  7.01 (0.00 / 0.00)
2. attack:  6.91 (0.47 / 0.00)
3. attack:  4.36 (0.72 / 0.22)
4. attack:  4.85 (0.78 / 0.34)

Total:     23.12


With this it seems that Dragon style and Jabbing style are pretty evened out when accounting for hit probabilities (and assuming that I did not do an error in my analysis). I find this pretty cool if it was actually an intended balancing choice.